In [153]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.utils import normalize
from tensorflow.keras.metrics import MeanIoU

In [154]:
#Resizing images, if needed
SIZE_X = 128 
SIZE_Y = 128
n_classes = 2

In [155]:
train_images_org = []

d = r"C:\Users\ikhlas\Documents\FYP\FYP-code\trainingData_semanticSegmentation\trainingData_semanticSegmentation\0_original_images"

for directory_path in glob.glob(d):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, 1)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images_org.append(img)

In [156]:
train_images = np.array(train_images_org, dtype=np.float32)

In [157]:

masks = [] 

d = r"C:\Users\ikhlas\Documents\FYP\FYP-code\trainingData_semanticSegmentation\trainingData_semanticSegmentation"
for directory_path in glob.glob(d):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        mask = cv2.imread(mask_path, flags=cv2.IMREAD_UNCHANGED)       
        # mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        masks.append(mask)

In [158]:
from tifffile import imread, imwrite
from skimage.transform import resize

masks = [] 

d = r"C:\Users\ikhlas\Documents\FYP\FYP-code\trainingData_semanticSegmentation\trainingData_semanticSegmentation"
for directory_path in glob.glob(d):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        img = imread(mask_path)
        mask = resize(img, (5, SIZE_X , SIZE_Y))
        masks.append(mask)

In [159]:
train_masks = np.array(masks, dtype=np.float32)

In [160]:
train_images.shape

(474, 128, 128, 3)

In [161]:
train_masks.shape

(474, 5, 128, 128)

In [162]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w, c = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w, c)

In [163]:
np.unique(train_masks_encoded_original_shape)

array([     0,      1,      2, ..., 211609, 211610, 211611], dtype=int64)

In [164]:
train_masks_encoded_original_shape = (train_masks_encoded_original_shape>0.5).astype(np.int)
np.unique(train_masks_encoded_original_shape)

array([0, 1])

In [165]:
train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=4)

In [166]:
train_masks_input.shape

(474, 5, 128, 128, 1)

In [167]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)

In [168]:
from tensorflow.keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=2)

In [169]:
test_masks_cat = to_categorical(y_test, num_classes=n_classes)

In [170]:
test_masks_cat.shape

(48, 5, 128, 128, 2)

In [172]:
X_train.shape

(426, 128, 128, 3)

In [171]:
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2]))

ValueError: cannot reshape array of size 69795840 into shape (426,5,128)

In [ ]:
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], 10))

In [ ]:
n_classes=4
activation='softmax'

LR = 0.0001
optim = tf.keras.optimizers.Adam(LR)

In [ ]:
dice_loss = sm.losses.DiceLoss(class_weights=np.array([0.25, 0.25, 0.25, 0.25])) 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

In [ ]:
metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [ ]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)

# preprocess input
X_train1 = preprocess_input1(X_train)
X_test1 = preprocess_input1(X_test)

# define model
model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

# compile keras model with defined optimozer, loss and metrics
model1.compile(optim, total_loss, metrics=metrics)

#model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

print(model1.summary())


history1=model1.fit(X_train1, 
          y_train_cat,
          batch_size=8, 
          epochs=50,
          verbose=1,
          validation_data=(X_test1, y_test_cat))


model1.save('res34_backbone_50epochs.hdf5')

In [ ]:
X_train1.shape

In [ ]:
X_test1.shape

In [ ]:
X_train.shape